In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
airports = pd.read_csv('/kaggle/input/flight-delays-prediction-challeng2021/airports.csv')
flights_train = pd.read_csv('/kaggle/input/flight-delays-prediction-challeng2021/flights_train.csv')

airports_origin = airports[['IATA_CODE','LATITUDE','LONGITUDE']].rename(columns = {'IATA_CODE' : 'ORIGIN_AIRPORT'})
airports_arrive = airports[['IATA_CODE','LATITUDE','LONGITUDE']].rename(columns = {'IATA_CODE' : 'DESTINATION_AIRPORT'})
flights_train = flights_train.merge(airports_origin, on = 'ORIGIN_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_origin', 'LONGITUDE' : 'LONGITUDE_origin'})
flights_train = flights_train.merge(airports_arrive, on = 'DESTINATION_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_arrival', 'LONGITUDE' : 'LONGITUDE_arrival'})

In [ ]:
station = "ATL"
url = f"https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={station}&data=tmpc&data=sknt&data=p01m&data=vsby&data=gust&data=skyc1&data=skyc2&data=skyc3&data=wxcodes&data=ice_accretion_6hr&data=snowdepth&year1=2015&month1=1&day1=1&year2=2015&month2=8&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=empty&trace=T&direct=no&report_type=1&report_type=2"
pd.read_csv(url)

In [7]:
clock_time_columns = [
    'SCHEDULED_DEPARTURE', # planned leaving time
    'DEPARTURE_TIME',      # actual leaving time
    'WHEELS_OFF',          # moment the airplane leaves the ground
    'SCHEDULED_ARRIVAL',   # planned arrical
]
relative_time_columns = [
    'TAXI_OUT',            # mintues between departure and wheels off
    'SCHEDULED_TIME',      # planned flight (+ground?) time
    'ARRIVAL_DELAY'        # difference between scheduled arrival and actual arrival
]

In [8]:
train = flights_train.drop(columns=[
#                             'TAXI_OUT',
#                             'WHEELS_OFF',
#                             'ARRIVAL_DELAY',
                            'LATITUDE_origin',
                            'LONGITUDE_origin',
                            'LATITUDE_arrival',
                            'LONGITUDE_arrival',
])

for x in train.columns:
        # set float columns to int
        if (train[x].dtype == 'float64'):
            train[x] = train[x].astype(np.int64)
        # fix clock time columns lol
        if x in clock_time_columns:
            train[x] = ((train[x] // 100)*60) + (train[x] % 100)

train["DEPARTURE_DELAY"]     = train.DEPARTURE_TIME - train.SCHEDULED_DEPARTURE
train["DEPARTURE_DELAY_COR"] = np.where(train.DEPARTURE_DELAY < -90, train.DEPARTURE_DELAY + 1440, train.DEPARTURE_DELAY)
    

In [ ]:
train.isna().sum()

In [4]:
plt.figure(figsize=(20, 10))
axis = sns.countplot(x=train['AIRLINE'], data=train, order=train['AIRLINE'].value_counts().index, orient="v")
axis.set_xticklabels(axis.get_xticklabels())
plt.title('TOTAL NUMBER OF FLIGHTS BY AIRLINE', fontsize=24)
plt.xlabel('AIRLINE')
plt.ylabel('NUMBER OF FLIGHTS')
plt.tight_layout()
plt.show()

In [9]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
ax = sns.scatterplot(data=train, y="ARRIVAL_DELAY", x="DEPARTURE_DELAY_COR", color = "#97D7F0")
ax.set_title('Arrival delay vs Corrected departure delay')
ax.set_ylabel('Arrival delay')
ax.set_xlabel('Corrected departure delay')

plt.show()


In [ ]:
sns.scatterplot(data=train, x="ARRIVAL_DELAY", y="SCHEDULED_DEPARTURE", hue="DISTANCE")

In [ ]:
sns.scatterplot(data=train, x="ARRIVAL_DELAY", y="DEPARTURE_TIME", hue="DISTANCE")

In [ ]:
#train.insert(10, "DEPDEL_DIST", (train["DEPARTURE_DELAY"] * train["DISTANCE"]))
train["DEPDEL_DIST"] = train["DEPARTURE_DELAY"] / train["DISTANCE"]

In [ ]:
sns.scatterplot(data=train, x="ARRIVAL_DELAY", y="DISTANCE")

In [ ]:
sns.scatterplot(data=train, x="ARRIVAL_DELAY", y="DEPDEL_DIST")

In [ ]:
sns.scatterplot(data=train, x="ARRIVAL_DELAY", y="DEPARTURE_DELAY", hue="DAY_OF_WEEK", alpha=0.5)

In [ ]:
sns.histplot(train, x="DISTANCE", bins = 30)

In [ ]:
train["DISTANCE"].describe()

In [ ]:
sns.pairplot(train)

# scheduled arrival y departure


In [ ]:
feature = train['SCHEDULED_ARRIVAL'] - train['SCHEDULED_DEPARTURE']
= train['ARRIVAL_DELAY']
plt.scatter(col, )

In [ ]:
feature = train['SCHEDULED_TIME']
target = train['ARRIVAL_DELAY']
plt.scatter(target, feature)

In [ ]:
fig = plt.figure(1, figsize=(15,9))
sns.set(font_scale=1.6)
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
sns.set_palette("mako")
ax = sns.barplot(x='ARRIVAL_DELAY', y='AIRLINE', color='#93DDDE', data=train.groupby('AIRLINE').ARRIVAL_DELAY.mean().sort_values(ascending=False).to_frame().reset_index())
ax.set_title('AVERAGE ARRIVAL DELAY BY AIRLINE (mins)')
ax.set_xlabel('AVERAGE DELAY BY AIRLINE (min)')
plt.axvline(x=train.ARRIVAL_DELAY.mean(), color='k', linestyle='--')
plt.axvline(x=0, color='k', linestyle='-')
plt.show()


In [ ]:
flights_t.ARRIVAL_DELAY.min()

In [ ]:
flights_t.YEAR.value_counts()

Looks like the year column is useless

In [ ]:
plt.figure(figsize=(20, 10))
flights_t.groupby('DAY_OF_WEEK').ARRIVAL_DELAY.mean().plot.bar()
plt.title('Average Flight delay per day', fontsize=18)
plt.xlabel('Day', fontsize=14)
plt.ylabel('Average delay', fontsize=14)
plt.axhline(y=flights_t.ARRIVAL_DELAY.mean(), color='r', linestyle='--')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
flights_t.groupby('DAY').ARRIVAL_DELAY.mean().plot.bar()
plt.title('Average Flight delay per Month', fontsize=18)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Number of Flights', fontsize=14)
plt.axhline(y=flights_t.ARRIVAL_DELAY.mean(), color='r', linestyle='--')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
flights_t.groupby('DAY').ARRIVAL_DELAY.mean().plot.bar()
plt.title('Average Flight delay per Month', fontsize=18)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Number of Flights', fontsize=14)
plt.axhline(y=flights_t.ARRIVAL_DELAY.mean(), color='r', linestyle='--')
plt.show()

In [ ]:

mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['hatch.linewidth'] = 1.8  

fig = plt.figure(1, figsize=(15,9))
ax = sns.barplot(x="DEPARTURE_DELAY", y="AIRLINE", data=train, color="#93DDDE", ci=None)
ax = sns.barplot(x="ARRIVAL_DELAY", y="AIRLINE", data=train, color="r", hatch = '///', alpha = 0.0, ci=None)
ax.yaxis.label.set_visible(False)
plt.title('DEPARTURE (blue) vs ARRIVAL (hatch) DELAY')
plt.xlabel('MEAN DEALY (min)', fontsize=14, labelpad=10);



In [ ]:
plt.figure(figsize=(20, 10))
flights_t.groupby('ORIGIN_AIRPORT').ARRIVAL_DELAY.mean().sort_values(ascending=False).plot.bar()
plt.title('Average Flight delay per origin', fontsize=18)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Number of Flights', fontsize=14)
plt.axhline(y=flights_t.ARRIVAL_DELAY.mean(), color='r', linestyle='--')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
flights_t.groupby('DESTINATION_AIRPORT').ARRIVAL_DELAY.mean().sort_values(ascending=False).plot.bar()
plt.title('Average Flight delay per destination', fontsize=18)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Number of Flights', fontsize=14)
plt.axhline(y=flights_t.ARRIVAL_DELAY.mean(), color='r', linestyle='--')
plt.show()